# Temperature and Rainfall Analysis using PyCaret - Regression

### Step 1: Merging the dataset

In [2]:
import pandas as pd

temperature_df = pd.read_excel('dataset/Temp_SRS_1Jan1964_to_31Oct2020.xlsx')
rainfall_df = pd.read_csv('dataset/Rainfall_at_SRS_30Nov1960_to_18Nov2020.csv')

rainfall_df = rainfall_df.fillna(0)

temperature_df['DATE'] = pd.to_datetime(temperature_df['DATE'], format='%Y-%m-%d')
rainfall_df['DATE'] = pd.to_datetime(rainfall_df['DATE'], format='%m/%d/%Y')

temperature_df['YEAR'] = pd.to_datetime(temperature_df.DATE).dt.year
rainfall_df['YEAR'] = pd.to_datetime(rainfall_df.DATE).dt.year

temperature_df_grouped = temperature_df.groupby('YEAR').agg({'LOW TEMP': 'mean', 'HIGH TEMP': 'mean'})
temperature_df_grouped['RAINFALL'] = rainfall_df.groupby('YEAR').agg({'200-F Rainfall (inches/day)': 'mean'})
rain_temp_df = pd.merge(temperature_df, rainfall_df, on='DATE')
temperature_df_grouped.reset_index(inplace=True)
temperature_df_grouped.head()
# print(rain_temp_df.head(10))

,YEAR,LOW TEMP,HIGH TEMP,RAINFALL
0,1964,52.980822,74.909589,0.196011
1,1965,52.030137,74.164384,0.123014
2,1966,51.624658,73.520548,0.131699
3,1967,52.408219,75.449315,0.116438
4,1968,53.278689,75.653005,0.094740


## Predicting rainfall using average yearly maximum and minimum temperatures using LSTM

In [14]:
from pycaret.time_series import *

train_df = temperature_df_grouped.query('YEAR < 2010')
test_df = temperature_df_grouped.query('YEAR >= 2010')

reg = setup(data=train_df, target='RAINFALL')

model = models()
